Load all packages you need.

In [1]:
import pandas as pd

Read the dataset that you prepared yesterday afternoon (without outlier and missing data). 

In [6]:
#Eigene Lösung
df_self = pd.read_pickle("../data/df_imputed_clean.pkl")
df_self.head()
df1 = df_self.drop(columns=['Mahalanobis_Distance','Multivariate_Outlier','Outlier'])
df1.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,148.0,72.0,35.0,169.0,33.6,0.627,50.0,1.0
1,1.0,85.0,66.0,29.0,58.6,26.6,0.351,31.0,0.0
2,8.0,183.0,64.0,25.8,164.6,23.3,0.672,32.0,1.0
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0,0.0
4,5.0,116.0,74.0,20.6,102.8,25.6,0.201,30.0,0.0


Let's try SOM on Age and Pregnancies. Don't forget the the variables should be normalized!

In [6]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from minisom import MiniSom
import matplotlib.pyplot as plt

# Select relevant columns
data = df1[["Age", "Pregnancies"]].values

# Normalize data
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data)

# Define SOM parameters
som_shape = (10, 10)  # Grid size (10x10 neurons)
input_len = data_normalized.shape[1]  # Number of features

# Initialize and train SOM
som = MiniSom(som_shape[0], som_shape[1], input_len, sigma=1.0, learning_rate=0.5)
som.random_weights_init(data_normalized)
som.train_random(data_normalized, 1000)  # Number of iterations

# Create scatter plot for the SOM result
plt.figure(figsize=(8, 8))
for x in data_normalized:
    w = som.winner(x)  # Get winning neuron coordinates
    plt.scatter(w[0], w[1], marker='o', color='blue', alpha=0.6)

plt.title("SOM Clustering of Age and Pregnancies")
plt.xlabel("SOM X")
plt.ylabel("SOM Y")
plt.grid(True)
plt.show()


AttributeError: partially initialized module 'pandas' has no attribute '_pandas_datetime_CAPI' (most likely due to a circular import)

Perform the SOM algorithm (with minisom a well documented package is available on conda-forge). Tian, Azarian & Pecht (2014) suggest that a good estimation for the number of nodes in your map should be nodes = 5*sqrt(datapoints). Lets start with a 11x11 map structure and and define rectangular neighbourhood (only vertical and horizontal, no diagonal neighbours). Use a random seed for for reproducibility.  <br>
Step 1:Initialize your SOM. <br>
Stel 2: Train your SOM on 1000 iterations. <br>


Step 3: Plot the neighbourdhood distance of your trained SOM. How many cluster do you detect?

Step 4: Run K-Means on the nodes of your SOM. <br>
Step 5: Assign each data point to it's BMU's cluster label.<br>
Step 6: Plot clusters on scatterplot of "Age" vs "Pregnancies". Compare the clusters with the clusters determined by the K-Means algorithm during the previous coding break. 